In [1]:
import sys
CODEBASE = "./codebase"
if CODEBASE not in sys.path:
    sys.path.append(CODEBASE)

from cooperative_evolution_optimizers import GrayBoxOptimizer
from differential_evolution import DifferentialEvolution as DE

from fitness_functions import FunctionFactory
get_sphere = FunctionFactory.get_sphere

import numpy as np
import matplotlib.pyplot as plt
from time import time

def plot_means_errs(xs, nparray):
    means = np.mean(nparray, axis=-1)
    errs = np.std(nparray, axis=-1)
    plt.errorbar(xs, means, yerr=errs, fmt="--o", capsize=5)

Define $P(x \leq VTR\ |\ N = n)$ as the probability that an optimizer with the sphere function of length $l$ reached fitness $x$ less than or equal to $VTR$ at generation $N = n$ and define $P_{GBO, k}\left(x \leq VTR\ \big|\ N = n\right)$ as the probability that an optimizer with CCGBO with the sphere function of length $l$ for each species reached a cumulative fitness of $x$ less than or equal to $VTR$ by generation $N = n$.

# Study the relationship between GBO and standard optimization

**First hypothesis** on fixed population size:

$$P_{GBO, k}(x \leq VTR\ |\ N = n) \approx P\left(x \leq \frac{VTR}{k}\ \big|\ N = n\right)^k$$

To test this and for varied $k$ we take several $n \leq N_{max}$ and estimate $P(x \leq VTR/k\ |\ N = n)$ over several runs. Then we also estimate $P_{GBO, k}(x \leq VTR\ |\ N = n)$ over several runs and compare the estimated probabilities, not forgetting to take the first one to the $k$-th power.

_I did some experiments with lenient VTRs but those were wrong because of some retarded indexing issues. After those I wrote:_

 > For every small $k$ the approximation doesn't hold, and for "bigger" $k$ we rather get the bound
 >
 > $$P_{GBO, k}(x \leq VTR\ |\ N = n) \geq P\left(x \leq \frac{VTR}{k}\ \big|\ N = n\right)^k$$
 >
 > This is because I have been using a rather "easy" $VTR$ and so when some of the species in the GBO hit the value $VTR/k$ before the $n$-th generation, they just keep on improving on behalf of the ones that haven't reached it yet, and so the cumulative fitness gets to $VTR$ earlier than in $n$ generations. If I reduce the $VTR$ a bit I suspect this will no longer happen, so that is what I will do now:

Changed $VTR$ to $10^{-10}$.

## Species of length 1, $1 \leq k \leq 10$, $VTR = 10^{-10}$.

In [9]:
Ks = list(range(1, 16+1))
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -10)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}


for k in Ks:
    estimated_quantiles = np.zeros(max_generations, dtype=np.double)
    gbo_quantiles = np.zeros(max_generations, dtype=np.double)
    print("k = {}".format(k))
    target_VTR = VTR/k
    ns = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
            
        de = DE(sphere, 1, goal_fitness=target_VTR,
                lower_bounds=LB, upper_bounds=UB, **de_args)
        while not de.has_converged():
            de.evolve()
        ns.append(de.generations())
    ns = np.array(ns)
    # compute some sort of quantiles
    # estimate the right n_k
    for n in range(max_generations):
        estimated_quantiles[n] = sum(ns <= n)/N
    
    print("Estimating the probability for the GBO")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[i] for i in range(k)],
                              train_partition = [[i] for i in range(k)],
                              lower_bounds = [LB]*1*k, upper_bounds = [UB]*1*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        # for some reason I need to shift this indexing...
        # probably one of the counters isn't right...
        q = gbo_quantiles[n+1]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

k = 1
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   3 ||    0.10 ->   0.100 :   0.20 (err  -0.100)
n =   4 ||    0.20 ->   0.200 :   0.50 (err  -0.300)
n =   5 ||    0.60 ->   0.600 :   1.10 (err  -0.500)
n =   6 ||    1.50 ->   1.500 :   2.20 (err  -0.700)
n =   7 ||    3.10 ->   3.100 :   3.50 (err  -0.400)
n =   8 ||    6.60 ->   6.600 :   8.10 (err  -1.500)
n =   9 ||   12.50 ->  12.500 :  13.30 (err  -0.800)
n =  10 ||   21.90 ->  21.900 :  23.30 (err  -1.400)
n =  11 ||   37.70 ->  37.700 :  40.10 (err  -2.400)
n =  12 ||   58.30 ->  58.300 :  59.80 (err  -1.500)
n =  13 ||   76.10 ->  76.100 :  77.20 (err  -1.100)
n =  14 ||   89.20 ->  89.200 :  89.20 (err   0.000)
n =  15 ||   96.80 ->  96.800 :  96.00 (err   0.800)
n =  16 ||   99.30 ->  99.300 :  99.20 (err   0.100)
n =  17 ||   99.90 ->  99.900 :  99.90 (err   0.000)
k = 2
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO


0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   5 ||    0.30 ->   0.000 :   0.00 (err   0.000)
n =   6 ||    0.50 ->   0.000 :   0.00 (err   0.000)
n =   7 ||    0.90 ->   0.000 :   0.00 (err   0.000)
n =   8 ||    2.10 ->   0.000 :   0.00 (err   0.000)
n =   9 ||    4.40 ->   0.000 :   0.00 (err   0.000)
n =  10 ||    8.60 ->   0.000 :   0.00 (err   0.000)
n =  11 ||   16.50 ->   0.000 :   0.00 (err   0.000)
n =  12 ||   27.90 ->   0.001 :   0.00 (err   0.001)
n =  13 ||   43.70 ->   0.058 :   2.30 (err  -2.242)
n =  14 ||   63.50 ->   1.679 :  17.70 (err -16.021)
n =  15 ||   79.60 ->  12.830 :  62.80 (err -49.970)
n =  16 ||   92.00 ->  47.216 :  91.20 (err -43.984)
n =  17 ||   97.30 ->  78.166 :  98.50 (err -20.334)
n =  18 ||   99.70 ->  97.332 :  99.70 (err  -2.368)
n =  19 ||   99.90 ->  99.104 : 100.00 (err  -0.896)
n =  20 ||  100.00 -> 100.000 : 100.00 (err   0.000)
k = 10
0 100 200 300 400 500 600 700

KeyboardInterrupt: 

The code above was going to run until $k \leq 16$ but because the $VTR$ was already at $10^{-10}$ that meant that the single DE would have to reach fitnesses under $10^{-11}$ which is possible, but it is not worth waiting for right now.

## The approximation does not hold

From a mathematical standpoint, we have that

$$P_{GBO, k}(x \leq VTR\ |\ N = n) \geq P(\cap_{i=1}^k \{x_i \leq \frac{VTR}{k} \}\ |\ N = n) = P(x \leq VTR/k\ |\ N = n)^k \tag{$Ineq_1$}$$

and we also have that

$$P_{GBO, k}(x \leq VTR\ |\ N = n) \leq P(\cap_{i=1}^k \{x_i \leq VTR \}\ |\ N = n) = P(x \leq VTR\ |\ N = n)^k \tag{$Ineq_2$}$$

and we were able to show that $Ineq_1$ isn't tight.

Now we will do something similar and check how tight $(Ineq_2)$ is.

## Species of length 1, $1 \leq k \leq 16$, $VTR = 10^{-10}$

In [10]:
Ks = list(range(1, 16+1))
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -10)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}

# first estimate probabilities for the simple optimizer
estimated_quantiles = np.zeros(max_generations, dtype=np.double)
ns = []
for i in range(10*N):
    if (i % 100) == 0:
        print(i, end = " ")
        
    de = DE(sphere, 1, goal_fitness=VTR,
           lower_bounds=LB, upper_bounds=UB, **de_args)
    while not de.has_converged():
        de.evolve()
    ns.append(de.generations())
ns = np.array(ns)
for n in range(max_generations):
    estimated_quantiles[n] = sum(ns <= n)/(10*N)

print("Estimating the probability for the GBOs")
for k in Ks:
    gbo_quantiles = np.zeros(max_generations, dtype=np.double)
    print("k = {}".format(k), end = " ")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[i] for i in range(k)],
                              train_partition = [[i] for i in range(k)],
                              lower_bounds = [LB]*1*k, upper_bounds = [UB]*1*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        q = gbo_quantiles[n+1]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 Estimating the probability for the GBOs
k = 1 0 100 200 300 400 500 600 700 800 900 
n =   1 ||    0.04 ->   0.040 :   0.00 (err   0.040)
n =   2 ||    0.11 ->   0.110 :   0.20 (err  -0.090)
n =   3 ||    0.35 ->   0.350 :   0.20 (err   0.150)
n =   4 ||    0.71 ->   0.710 :   0.50 (err   0.210)
n =   5 ||    1.30 ->   1.300 :   1.10 (err   0.200)
n =   6 ||    2.35 ->   2.350 :   1.90 (err   0.450)
n =   7 ||    4.18 ->   4.180 :   4.30 (err  -0.120)
n =   8 ||    7.60 ->   7.600 :   7.90 (err  -0.300)
n =

k = 9 0 100 200 300 400 500 600 700 800 900 
n =   1 ||    0.04 ->   0.000 :   0.00 (err   0.000)
n =   2 ||    0.11 ->   0.000 :   0.00 (err   0.000)
n =   3 ||    0.35 ->   0.000 :   0.00 (err   0.000)
n =   4 ||    0.71 ->   0.000 :   0.00 (err   0.000)
n =   5 ||    1.30 ->   0.000 :   0.00 (err   0.000)
n =   6 ||    2.35 ->   0.000 :   0.00 (err   0.000)
n =   7 ||    4.18 ->   0.000 :   0.00 (err   0.000)
n =   8 ||    7.60 ->   0.000 :   0.00 (err   0.000)
n =   9 ||   13.56 ->   0.000 :   0.00 (err   0.000)
n =  10 ||   23.64 ->   0.000 :   0.00 (err   0.000)
n =  11 ||   39.15 ->   0.022 :   0.00 (err   0.022)
n =  12 ||   58.10 ->   0.754 :   0.00 (err   0.754)
n =  13 ||   76.94 ->   9.449 :   2.00 (err   7.449)
n =  14 ||   90.43 ->  40.440 :  19.50 (err  20.940)
n =  15 ||   97.34 ->  78.455 :  64.70 (err  13.755)
n =  16 ||   99.33 ->  94.129 :  92.90 (err   1.229)
n =  17 ||   99.81 ->  98.303 :  99.50 (err  -1.197)
n =  18 ||   99.98 ->  99.820 :  99.90 (err  -0.080)
n